## Tensorflow inference to generate heatmap of crowd

Handling Imports

In [149]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
import os
import random
import math
import sys
from pyheatmap.heatmap import HeatMap

Loading Model and Weights from disk

In [150]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = keras.models.model_from_json(loaded_model_json)

model.load_weights("weights.h5")

print("Loaded model from disk successfully")

Loaded model from disk successfully


Setting up paths of image and other things

In [156]:
input_image = "/home/hardik/Projects/Minor-Project/new/IMG_359.jpg"

Loading and preprocessing input image to correspond to input layer size

In [157]:
img = np.array(cv2.imread(input_image, 0))
img = (img - 127.5) / 128
inputs = np.reshape(img, [1, 768, 1024, 1])


Inferencing through model

In [158]:
outputs = model.predict(inputs)
predicted_count = np.sum(outputs)

1/1 [==============================] - 0s 222ms/step


Printing predicted count

In [159]:
print("Predicted Count:", predicted_count)

Predicted Count: 61.01952


Generating density numpy array

In [160]:
y_p_den = np.reshape(outputs, (outputs.shape[1], outputs.shape[2]))

Generating Heatmap

In [161]:
den_resized = np.zeros((y_p_den.shape[0] * 4, y_p_den.shape[1] * 4))
for i in range(den_resized.shape[0]):
    for j in range(den_resized.shape[1]):
        den_resized[i][j] = y_p_den[int(i / 4)][int(j / 4)] / 16
den = den_resized
count = np.sum(den)
den = den * 10 / np.max(den)

crowd_img = cv2.imread(input_image, 1)

data = []
pts = []

for j in range(len(den)):
    for i in range(len(den[0])):
        pts.append([i + 1, j + 1])
        for k in range(int(den[j][i])):
            data.append([i + 1, j + 1])

hm = HeatMap(data, base=input_image)
hm.heatmap(save_as = 'output.png')
print("Heatmap Generated to output.png")

Heatmap Generated to output.png
